# Пакет `pandas`: группировка, агрегирование, сводные таблицы

---

**Источники:**

[Сводные таблицы в Python](http://datareview.info/article/svodnyie-tablitsyi-v-python/)

---

## Подготовка окружения

In [1]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

# для Linux
!which pip

# для Windows
# !pip -V

/home/ira/anaconda3/envs/LevelUp_DataScience/bin/pip


In [2]:
!conda install pandas -y

Solving environment: done

# All requested packages already installed.



In [3]:
import pandas as pd

pd.__version__

'1.2.1'

## Группировка и агрегирование

За группировку отвечает метод `pandas.groupby`.

Для примера возьмем [данные (data set) Титаника](https://www.kaggle.com/c/titanic).

In [4]:
titanic_df = pd.read_csv('./../../data/titanic.csv')

# показать все колонки
pd.options.display.max_columns = None

titanic_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
# показать первые 5 строк
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# подсчитать, сколько женщин и мужчин выжило, а сколько нет
titanic_df.groupby(['Sex', 'Survived'])['PassengerId'].count()

Sex     Survived
female  0            81
        1           233
male    0           468
        1           109
Name: PassengerId, dtype: int64

In [7]:
# анализ выживания в разрезе класса кабины
titanic_df.groupby(['Pclass', 'Survived'])['PassengerId'].count()

Pclass  Survived
1       0            80
        1           136
2       0            97
        1            87
3       0           372
        1           119
Name: PassengerId, dtype: int64

In [8]:
# процент выживших для каждого пола
titanic_df.groupby('Sex')[['Survived']].mean()

,Survived
Sex,
female,0.742038
male,0.188908


Грубо говоря, из каждых четырех женщин, находившихся на борту, выжили три, в то время как из каждых пяти мужчин выжил только один!

In [9]:
# взаимосвязь между показателем выживаемости, полом и классом
titanic_df.groupby(['Sex', 'Pclass'])['Survived'].aggregate('mean').unstack()

Pclass,1,2,3
Sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [10]:
# если без unstack()
titanic_df.groupby(['Sex', 'Pclass'])['Survived'].aggregate('mean')

Sex     Pclass
female  1         0.968085
        2         0.921053
        3         0.500000
male    1         0.368852
        2         0.157407
        3         0.135447
Name: Survived, dtype: float64

Хотя каждый шаг этой последовательности вполне понятен, тем не менее длинную строку кода достаточно трудно читать и использовать.

Подобные операции широко распространены, в связи с чем библиотека `pandas` имеет в своем составе специальный метод `pivot_table,` лаконично реализующий данный тип многомерной агрегации.

## Что такое сводная таблица?

**Сводная таблица (англ. Pivot table)** — инструмент обработки данных, служащий для их обобщения.


*Термин "сводная таблица" может быть знаком из `Microsoft Excel` или любым иным, предназначенным для обработки и анализа данных. 


В `pandas` сводные таблицы строятся через метод `DataFrame.pivot_table`.

In [11]:
# взаимосвязь между показателем выживаемости, полом и классом
titanic_df.pivot_table(values='Survived', 
                       index='Sex', 
                       columns='Pclass')

Pclass,1,2,3
Sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [12]:
# посчитать сколько всего женщин и мужчин было в конкретном классе корабля
pvt = titanic_df.pivot_table(index=['Sex'], 
                             columns=['Pclass'], 
                             values='Name', 
                             aggfunc='count')

pvt

Pclass,1,2,3
Sex,,,
female,94,76,144
male,122,108,347


In [13]:
# качестве индекса будет пол человека, 
# колонками станут значения из Pclass, 
# функцией агрегирования будет count по колонке Name.
pvt.loc['female', [1, 2, 3]]

Pclass
1     94
2     76
3    144
Name: female, dtype: int64

## Многоуровневые сводные таблицы

Точно так же, как при использовании `groupby`, группирование в сводной таблице может иметь несколько уровней и задаваться посредством различных параметров.

In [14]:
# разделить возраст на интервалы
age = pd.cut(titanic_df['Age'], [0, 18, 80])

age

0      (18.0, 80.0]
1      (18.0, 80.0]
2      (18.0, 80.0]
3      (18.0, 80.0]
4      (18.0, 80.0]
           ...     
886    (18.0, 80.0]
887    (18.0, 80.0]
888             NaN
889    (18.0, 80.0]
890    (18.0, 80.0]
Name: Age, Length: 891, dtype: category
Categories (2, interval[int64]): [(0, 18] < (18, 80]]

In [15]:
# взаимосвязь между 
# показателем выживаемости, полом, классом и возрастом

titanic_df.pivot_table(values='Survived', 
                       index=['Sex', age], 
                       columns='Pclass')

Pclass                  1         2         3
Sex    Age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

In [16]:
# добавить информацию о стоимости билета (квантили):
fare = pd.qcut(titanic_df['Fare'], 2)

fare

0       (-0.001, 14.454]
1      (14.454, 512.329]
2       (-0.001, 14.454]
3      (14.454, 512.329]
4       (-0.001, 14.454]
             ...        
886     (-0.001, 14.454]
887    (14.454, 512.329]
888    (14.454, 512.329]
889    (14.454, 512.329]
890     (-0.001, 14.454]
Name: Fare, Length: 891, dtype: category
Categories (2, interval[float64]): [(-0.001, 14.454] < (14.454, 512.329]]

In [17]:
titanic_df.pivot_table(values='Survived', 
                       index=['Sex', age], 
                       columns=[fare, 'Pclass'])

Fare            (-0.001, 14.454]                     (14.454, 512.329]  \
Pclass                         1         2         3                 1   
Sex    Age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 80]              NaN  0.880000  0.444444          0.972973   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 80]              0.0  0.098039  0.125000          0.391304   

Fare                                 
Pclass                  2         3  
Sex    Age                           
female (0, 18]   1.000000  0.318182  
       (18, 80]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 80]  0.030303  0.192308

В результате получили четырехмерную агрегацию, демонстрирующую взаимосвязь между соответствующими величинами.

## Дополнительные параметры сводной таблицы

Полная [сигнатура метода `pivot_table`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) объекта `DataFrame` является следующей:

```
pandas.pivot_table(values=None, 
                   index=None, 
                   columns=None, 
                   aggfunc='mean', 
                   fill_value=None, 
                   margins=False, 
                   dropna=True, 
                   margins_name='All', 
                   observed=False)
```


Параметры **`fill_value`** и **`dropna`** задают способ обработки отсутствующих данных. 

Параметр **`aggfunc`** задает тип агрегации. По умолчанию его значение равно `mean`. Как и в случае `groupby`, тип агрегации можно задать либо с помощью предопределенной строки (например, `sum`, `mean`, `count`, `min`, `max` и др.), либо посредством функции, реализующей агрегацию (например, `np.sum()`, `min()`, `sum()` и др.).

Кроме того, параметр **`aggfunc`** может быть задан в виде словаря, *отображающего* столбцы на любые из желаемых значений, перечисленных выше:

In [18]:
titanic_df.pivot_table(index='Sex', 
                       columns='Pclass', 
                       aggfunc={'Survived': sum, 
                                'Fare': 'mean'})

# в данном случае не задается параметр values, 
# так как values задается автоматически, 
# когда параметр aggfunc представлен в виде отображения

Fare                       Survived        
Pclass           1          2          3        1   2   3
Sex                                                      
female  106.125798  21.970121  16.118810       91  70  72
male     67.226127  19.741782  12.661633       45  17  47

С помощью параметра **`margins`** можно вычислить обобщенные значения по каждой группе.

In [19]:
titanic_df.pivot_table(values='Survived', 
                    index='Sex', 
                    columns='Pclass', 
                    margins=True)

Pclass,1,2,3,All
Sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838


Представленный выше код автоматически дает нам процент выживших в зависимости от пола без учета класса, в зависимости от класса без учета пола, а также общий процент выживших, составляющий ~38%.